<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1


    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

Matplotlib is building the font cache; this may take a moment.


In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-15 15:20:08
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16


-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.40 C
-------------------
Today PnL: 1.26 L (0.91%)
Current PnL: -21.94 L (-14.62%)
CY Booked + Current PnL: -10.04 L (-6.69%)
-------------------
Total profit:  1.63 L
Total loss:  -23.57 L
-------------------
Total Booked + Current PnL: 16.94 L (13.76%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.53%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 89.13 L (63.87%)
Deployed:  1.23 C
Current:  1.40 C
CAGR/XIRR %: 7.45%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CIPLA,1662.74,-20.12,56.0,X-LC,3.71,213364.0,8864.0,14423.0,0.33,4.33,6.76,11.39,10.0,0.61,1.54,13.67,X40N,ATH,PHARMA
76,TTKPRESTIG,770.00,95.52,47.0,M-SC,1.58,84240.0,-16537.0,16629.0,-0.36,-16.41,19.74,0.09,245.0,-0.99,0.61,10.67,OX40N,NTT,DURABLES
33,ICICIGI,2260.25,-12.91,68.0,X-MC,1.77,149894.0,13761.0,17358.0,9.21,10.11,11.58,22.86,91.0,0.79,1.08,24.00,X40,ATH,INSURANCE
83,VOLTAS,1530.00,0.88,57.0,X-MC,1.10,211770.0,20028.0,17725.0,1.44,10.45,8.37,19.69,99.0,1.13,1.53,17.95,XY25,NTT,AC
56,RELIANCE,1533.00,-14.41,47.0,X-LC,2.97,214453.0,4087.0,24705.0,-0.09,1.94,11.52,13.68,37.0,0.17,1.55,18.79,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,ICICIGI,2260.25,-12.91,68.0,X-MC,1.77,149894.0,13761.0,17358.0,9.21,10.11,11.58,22.86,91.0,0.79,1.08,24.00,X40,ATH,INSURANCE
67,STARHEALTH,761.00,19.30,67.0,H-SC,8.41,265826.0,-15297.0,148171.0,4.54,-5.44,55.74,47.26,144.0,-0.10,1.92,42.42,XY24,NTT,INSURANCE
55,RELAXO,1176.00,-38.81,46.0,X-SC,3.56,83397.0,-61763.0,137688.0,3.89,-42.55,165.10,52.31,136.0,-0.45,0.60,11.63,X40N,NTT,FOOTWEAR
49,NESTLEIND,1377.00,-7.20,62.0,X-LC,2.39,285597.0,20171.0,36614.0,3.83,7.60,12.82,21.40,11.0,0.55,2.06,14.75,XY25,NTT,FMCG
81,VALIANTORG,838.00,-284.65,32.0,H-SC,3.50,102663.0,-60942.0,182258.0,3.80,-37.25,177.53,74.15,139.0,-0.33,0.74,31.60,XR,NTT,CHEMICALS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,INDIGOPNTS,1408.0,93.64,29.0,M-SC,3.36,126852.0,-47707.0,47734.0,-1.88,-27.33,37.63,0.02,221.0,-1.00,0.92,10.34,OX40N,NTT,PAINTS
46,LAOPALA,464.0,95.46,30.0,H-SC,1.51,89404.0,-41701.0,92953.0,-1.62,-31.81,103.97,39.09,142.0,-0.45,0.65,15.14,AR,NTT,CERAMICS
13,BERGEPAINT,680.0,-20.78,45.0,X-MC,2.20,212888.0,-14451.0,62504.0,-1.55,-6.36,29.36,21.14,106.0,-0.23,1.54,19.99,XY24,NTT,PAINTS
34,ICICIPRULI,790.0,-22.60,41.0,X-MC,2.08,133103.0,-2685.0,45441.0,-1.52,-1.98,34.14,31.48,107.0,-0.06,0.96,9.90,X40,ATH,INSURANCE
38,INDUSINDBK,1800.0,243.84,46.0,L-MC,11.22,42197.0,-35010.0,60401.0,-1.35,-45.35,143.14,32.89,258.0,-0.58,0.30,16.21,XR,NTT,BANKS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-186.36,68.0,M-SC,17.34,148698.0,-522.0,144698.0,0.27,-0.35,97.31,96.62,208.0,-0.0,1.07,60.92,XY25,NTT,FINANCE
59,SAIL,228.00,41.94,47.0,M-MC,12.28,224273.0,-689.0,168339.0,1.14,-0.31,75.06,74.53,192.0,-0.0,1.62,31.85,XY24,BTT,STEEL
36,INDIAMART,4810.62,-55.76,36.0,H-SC,8.26,123326.0,-10.0,131638.0,-0.34,-0.01,106.74,106.72,119.0,-0.0,0.89,23.37,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,107.70,60.0,H-SC,11.56,131166.0,-11313.0,30837.0,2.10,-7.94,23.51,13.70,163.0,-0.37,0.95,54.81,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,95.52,47.0,M-SC,1.58,84240.0,-16537.0,16629.0,-0.36,-16.41,19.74,0.09,245.0,-0.99,0.61,10.67,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.97,48.0,H-SC,1.84,220563.0,-49104.0,85446.0,-0.15,-18.21,38.74,13.47,138.0,-0.57,1.59,12.15,XY24,NTT,PAINTS
17,CERA,9475.0,-23.63,41.0,H-SC,2.11,139184.0,-36719.0,78736.0,-0.65,-20.87,56.57,23.89,149.0,-0.47,1.01,20.20,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-24.55,44.0,H-MC,7.01,102040.0,-29795.0,72020.0,0.67,-22.60,70.58,32.03,98.0,-0.41,0.74,14.29,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,47.87,51.0,M-LC,6.30,206745.0,3575.0,38558.0,1.51,1.76,18.65,20.74,55.0,0.09,1.49,7.40,XY25,NTT,FINANCE
31,HINDZINC,730.22,36.83,69.0,M-LC,8.93,223416.0,18340.0,94237.0,1.57,8.94,42.18,54.89,52.0,0.19,1.61,35.19,X5K,ATH,METALS
85,WIPRO,420.00,-11.52,56.0,M-LC,5.76,155267.0,4322.0,105131.0,0.81,2.86,67.71,72.51,53.0,0.04,1.12,9.06,XR,NTT,IT
78,UNIONBANK,163.00,-11.63,56.0,M-LC,9.53,158691.0,17851.0,26644.0,1.88,12.67,16.79,31.59,66.0,0.67,1.15,42.52,XY24,NTT,BANKS
11,BANKINDIA,190.00,-24.22,63.0,H-MC,12.74,194388.0,14580.0,97835.0,1.68,8.11,50.33,62.52,88.0,0.15,1.40,44.96,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,36.83,69.0,M-LC,8.93,223416.0,18340.0,94237.0,1.57,8.94,42.18,54.89,52.0,0.19,1.61,35.19,X5K,ATH,METALS
11,BANKINDIA,190.00,-24.22,63.0,H-MC,12.74,194388.0,14580.0,97835.0,1.68,8.11,50.33,62.52,88.0,0.15,1.40,44.96,XR,NTT,BANKS
85,WIPRO,420.00,-11.52,56.0,M-LC,5.76,155267.0,4322.0,105131.0,0.81,2.86,67.71,72.51,53.0,0.04,1.12,9.06,XR,NTT,IT
36,INDIAMART,4810.62,-55.76,36.0,H-SC,8.26,123326.0,-10.0,131638.0,-0.34,-0.01,106.74,106.72,119.0,-0.00,0.89,23.37,AR,ATH,MISC
84,WHIRLPOOL,2270.00,-47.48,39.0,M-SC,3.96,88748.0,-2750.0,81506.0,2.54,-3.01,91.84,86.07,223.0,-0.03,0.64,28.88,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-55.15,12.0,X-LC,3.34,142260.0,-134263.0,244332.0,-0.90,-48.55,171.75,39.81,54.0,-0.55,1.03,0.00,XY24,NTT,AUTO
12,BATAINDIA,2096.00,-37.44,35.0,X-SC,4.41,93450.0,-35220.0,80517.0,0.15,-27.37,86.16,35.20,219.0,-0.44,0.67,7.26,X40,NTT,FOOTWEAR
51,QUESS,986.00,-47.66,35.0,X-SC,38.37,52144.0,-12862.0,163790.0,1.69,-19.79,314.11,232.18,198.0,-0.08,0.38,1.69,XY24,NTT,MISC
28,HAVELLS,2069.16,-7.88,35.0,X-MC,1.50,228493.0,-19238.0,94299.0,1.08,-7.77,41.27,30.30,92.0,-0.20,1.65,4.49,X40,ATH,ELECTRICAL
22,DABUR,735.00,-11.58,40.0,X-MC,3.42,239226.0,-8842.0,117245.0,1.21,-3.56,49.01,43.70,102.0,-0.08,1.73,10.37,XY24,BTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-2.83,45.0,H-LC,0.92,156005.0,-30090.0,77566.0,0.41,-16.17,49.72,25.51,15.0,-0.39,1.13,14.96,AR,ATH,ELECTRICAL
83,VOLTAS,1530.00,0.88,57.0,X-MC,1.10,211770.0,20028.0,17725.0,1.44,10.45,8.37,19.69,99.0,1.13,1.53,17.95,XY25,NTT,AC
28,HAVELLS,2069.16,-7.88,35.0,X-MC,1.50,228493.0,-19238.0,94299.0,1.08,-7.77,41.27,30.30,92.0,-0.20,1.65,4.49,X40,ATH,ELECTRICAL
33,ICICIGI,2260.25,-12.91,68.0,X-MC,1.77,149894.0,13761.0,17358.0,9.21,10.11,11.58,22.86,91.0,0.79,1.08,24.00,X40,ATH,INSURANCE
49,NESTLEIND,1377.00,-7.20,62.0,X-LC,2.39,285597.0,20171.0,36614.0,3.83,7.60,12.82,21.40,11.0,0.55,2.06,14.75,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-55.15,12.0,X-LC,3.34,142260.0,-134263.0,244332.0,-0.90,-48.55,171.75,39.81,54.0,-0.55,1.03,0.00,XY24,NTT,AUTO
51,QUESS,986.00,-47.66,35.0,X-SC,38.37,52144.0,-12862.0,163790.0,1.69,-19.79,314.11,232.18,198.0,-0.08,0.38,1.69,XY24,NTT,MISC
82,VBL,671.64,-21.10,41.0,X-LC,10.66,284101.0,-31741.0,144409.0,0.84,-10.05,50.83,35.67,5.0,-0.22,2.05,2.41,X40N,ATH,FMCG
41,ITC,452.00,-39.61,44.0,X-LC,2.82,195600.0,-4538.0,25428.0,0.81,-2.27,13.00,10.44,4.0,-0.18,1.41,3.17,X40,NTT,FMCG
73,TCS,4389.97,-29.97,45.0,X-LC,12.33,279180.0,-66776.0,133476.0,0.70,-19.30,47.81,19.28,1.0,-0.50,2.02,3.21,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.66,35.0,X-SC,38.37,52144.0,-12862.0,163790.0,1.69,-19.79,314.11,232.18,198.0,-0.08,0.38,1.69,XY24,NTT,MISC
55,RELAXO,1176.00,-38.81,46.0,X-SC,3.56,83397.0,-61763.0,137688.0,3.89,-42.55,165.10,52.31,136.0,-0.45,0.60,11.63,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-37.44,35.0,X-SC,4.41,93450.0,-35220.0,80517.0,0.15,-27.37,86.16,35.20,219.0,-0.44,0.67,7.26,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-5.38,49.0,X-SC,1.16,107444.0,5440.0,38486.0,1.52,5.33,35.82,43.06,122.0,0.14,0.78,23.36,X40N,ATH,MISC
32,HONAUT,58357.33,-23.00,45.0,X-SC,2.30,108495.0,-19443.0,66573.0,0.21,-15.20,61.36,36.84,143.0,-0.29,0.78,11.07,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-29.97,45.0,X-LC,12.33,279180.0,-66776.0,133476.0,0.70,-19.30,47.81,19.28,1.0,-0.50,2.02,3.21,X40,ATH,IT
39,INFY,2275.00,-19.24,47.0,X-LC,7.64,314047.0,1159.0,170528.0,-1.04,0.37,54.30,54.87,3.0,0.01,2.27,7.04,X40,BTT,IT
41,ITC,452.00,-39.61,44.0,X-LC,2.82,195600.0,-4538.0,25428.0,0.81,-2.27,13.00,10.44,4.0,-0.18,1.41,3.17,X40,NTT,FMCG
82,VBL,671.64,-21.10,41.0,X-LC,10.66,284101.0,-31741.0,144409.0,0.84,-10.05,50.83,35.67,5.0,-0.22,2.05,2.41,X40N,ATH,FMCG
1,ABB,7934.00,-40.55,47.0,H-LC,7.42,248472.0,-13147.0,132361.0,1.11,-5.03,53.27,45.57,7.0,-0.10,1.79,5.73,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6611.11,52.0,L-SC,6.06,73645.0,-20165.0,96784.0,3.68,-21.50,131.42,81.67,269.0,-0.21,0.53,42.62,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,49.0,L-SC,28.70,81043.0,-32506.0,72558.0,-0.09,-28.63,89.53,35.27,268.0,-0.45,0.59,98.36,XR,NTT,HOTELS
48,MASFIN,398.61,-20.05,44.0,H-SC,9.88,90630.0,-7350.0,28956.0,0.83,-7.50,31.95,22.05,152.0,-0.25,0.65,31.74,XR,ATH,FINANCE
77,UJJIVANSFB,60.00,107.70,60.0,H-SC,11.56,131166.0,-11313.0,30837.0,2.10,-7.94,23.51,13.70,163.0,-0.37,0.95,54.81,OX40N,NTT,BANKS
68,SURYODAY,240.00,57.89,54.0,H-SC,9.94,145146.0,-33925.0,99657.0,-0.18,-18.95,68.66,36.71,135.0,-0.34,1.05,44.01,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-186.36,68.0,M-SC,17.34,148698.0,-522.0,144698.0,0.27,-0.35,97.31,96.62,208.0,-0.00,1.07,60.92,XY25,NTT,FINANCE
68,SURYODAY,240.00,57.89,54.0,H-SC,9.94,145146.0,-33925.0,99657.0,-0.18,-18.95,68.66,36.71,135.0,-0.34,1.05,44.01,XR,NTT,BANKS
31,HINDZINC,730.22,36.83,69.0,M-LC,8.93,223416.0,18340.0,94237.0,1.57,8.94,42.18,54.89,52.0,0.19,1.61,35.19,X5K,ATH,METALS
67,STARHEALTH,761.00,19.30,67.0,H-SC,8.41,265826.0,-15297.0,148171.0,4.54,-5.44,55.74,47.26,144.0,-0.10,1.92,42.42,XY24,NTT,INSURANCE
4,ANGELONE,3033.00,13.27,66.0,X-SC,9.88,200580.0,9574.0,48119.0,1.73,5.01,23.99,30.21,157.0,0.20,1.45,26.94,X40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.54
1,25,44.01
2,50,75.89


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.42
LC    33.70
MC    21.90
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.63
X40      14.82
XY25     12.16
XR       11.00
X40N     10.17
AR        8.19
OX40N     7.67
X200      1.82
X5K       1.61
SR        1.13
MH        0.82
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.63
X-LC    22.56
X-MC    16.15
M-SC    12.67
M-LC     5.37
H-LC     4.74
X-SC     4.66
H-MC     3.83
M-MC     1.62
L-SC     1.46
L-LC     1.03
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.08,-5.26,39.95
IT,12.99,-16.94,80.19
FINANCE,12.21,-11.77,62.43
MISC,6.92,-20.30,86.96
BANKS,6.43,-12.11,71.08
PAINTS,5.56,-20.05,37.95
ELECTRICAL,5.35,-11.05,50.01
INSURANCE,3.96,-0.77,38.44
AUTO,2.81,-47.66,118.25


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3335687.0,22
XR,1386311.0,14
AR,1195430.0,9
X40,786609.0,10
X40N,619994.0,8
OX40N,586969.0,10
XY25,498328.0,8
SR,265163.0,2
X5K,94237.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3454912.0,25
M-SC,1286800.0,15
X-LC,1117009.0,13
X-MC,1091313.0,11
X-SC,535173.0,6
H-MC,350079.0,3
H-LC,282734.0,3
M-LC,264570.0,4
L-SC,259412.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1200936.0      6
           XR         822303.0      7
           AR         821624.0      5
M-SC       XY24       695819.0      6
X-MC       XY24       591707.0      4
X-LC       X40        494876.0      5
           XY24       308228.0      2
X-SC       X40N       290866.0      4
H-SC       OX40N      273774.0      4
           SR         265163.0      2
X-LC       X40N       212436.0      3
X-MC       X40        211216.0      4
H-LC       AR         209927.0      2
H-MC       XY24       180224.0      1
X-MC       XY25       171698.0      2
L-SC       XR         169342.0      2
M-MC       XY24       168339.0      1
M-SC       AR         163879.0      2
X-SC       XY24       163790.0      1
M-SC       OX40N      151105.0      4
           XY25       144698.0      1
           XR         131299.0      2
X-MC       X40N       116692.0      1
M-LC       XR         105131.0      1
X-LC       XY25       101469.0      3
H-MC       XR          97835.0      1
M-LC       X5K         94237.0      1
L-SC       OX40N       90070.0      1
X-SC       X40         80517.0      1
H-LC       X200        72807.0      1
H-MC       OX40N       72020.0      1
H-SC       MH          71112.0      1
L-MC       XR          60401.0      1
L-LC       XY25        41905.0      1
M-LC       XY25        38558.0      1
           XY24        26644.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 39.0 seconds
